In [1]:
%matplotlib inline

import subprocess
import itertools

import numpy as np
import requests
import pytz
import netCDF4
from osgeo import gdal
from os import path
from osgeo.gdalconst import *
from tqdm import tqdm
from bs4 import BeautifulSoup


In [2]:
url = 'http://opendap.deltares.nl/thredds/dodsC/opendap/rijkswaterstaat/kusthoogte/catalog.html'
ext = 'nc'
urls = []

def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

for ncfile in listFD(url, ext):
    items = ncfile.split('/catalog.html/')
    filename = items[1].split('/')[-1]
    url = items[0] + '/' + filename
    urls.append(url)

In [3]:
grids = []
for url in tqdm(urls[1:]):
    ds = netCDF4.Dataset(url)
    times = netCDF4.num2date(ds.variables['time'][:], ds.variables['time'].units)
    local = pytz.timezone("Europe/Amsterdam")
    times = [local.localize(t, is_dst=None).astimezone(pytz.utc) for t in times]
    arrs = []
    z = ds.variables['z'][:]
    x = ds.variables['x'][:]
    y = ds.variables['y'][:]
    
    grids.append({
        "url": url,
        "x": x,
        "y": y,
        "z": z,
        "times": times
    })
    ds.close()


100%|████████████████████████████████████████| 220/220 [11:14<00:00,  5.60s/it]


In [6]:
count = len(list(itertools.chain.from_iterable([g['times'] for g in grids])))
count

2207

In [7]:
print(grids[0]['z'][0])

[[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]]


In [8]:
ee_collection_path = 'users/gena/eo-bathymetry/rws_lidar'

In [9]:
def run(cmd):
    # print(cmd)
    subprocess.call(cmd)

In [10]:
i = 0
start_index = 0
for g in tqdm(grids):
    ncols = len(g['x'])
    nrows = len(g['y'])
    cellsize = g['x'][1] - g['x'][0]
    xllcorner = np.min(g['x'])
    yllcorner = np.min(g['y'])
    nodata_value = -32767
    z = g['z']

    for it, t in enumerate(g['times']):
        if i < start_index:
            i = i + 1
            continue
        
        filename = 'bathymetry_' + str(i).rjust(3, '0')
        filepath = '../output/' + filename
        filepath_asc = filepath + '.asc'
        filepath_tif = filepath + '.tif'
        filename_tif = filename + '.tif'

        zi = z[it]

        with open(filepath_asc, 'w') as f:
            f.write('ncols {0}\n'.format(ncols))
            f.write('nrows {0}\n'.format(nrows))
            f.write('cellsize {0}\n'.format(cellsize))
            f.write('xllcorner {0}\n'.format(xllcorner))
            f.write('yllcorner {0}\n'.format(yllcorner))
            f.write('nodata_value {0}\n'.format(nodata_value))
            for row in range(nrows):
                s = ' '.join([str(v) for v in zi[row,]]).replace('--', str(nodata_value))
                f.write(s)
                f.write('\n')

        cmd = 'gdal_translate -ot Float32 -a_srs EPSG:28992 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=6 -of GTiff {0} {1}'\
            .format(filepath_asc, filepath_tif)
        run(cmd)
        
        filepath_gs = 'gs://hydro-earth/sandengine_lidar/' + filename_tif
        
        cmd = 'gsutil cp {0} {1}'\
            .format(filepath_tif, filepath_gs)
        run(cmd)
        
        filepath_ee = ee_collection_path + '/' + filename        
        cmd = 'earthengine upload image --wait --asset_id={0} --nodata_value={1} {2}'\
            .format(filepath_ee, nodata_value, filepath_gs)        
        run(cmd)
        
        time_start = int(grids[0]['times'][0].timestamp() * 1000)
        cmd = 'earthengine asset set --time_start {0} {1}'\
            .format(time_start, filepath_ee)
        run(cmd)

        cmd = 'earthengine acl set public {0}'\
            .format(filepath_ee)
        run(cmd)
        
        i = i + 1

  7%|██▍                                | 15/220 [1:23:35<34:40:14, 608.85s/it]

KeyboardInterrupt: 

In [ ]:
        filepath_gs = 'gs://hydro-earth/sandengine_lidar/' + filename_tif
        
        gsutil = 'D:/src/google-cloud-sdk/bin/gsutil.cmd' # relative path is not defined on Windows
        cmd = gsutil + ' cp {0} {1}'\
            .format(filepath_tif, filepath_gs)
        run(cmd)
        
        filepath_ee = ee_collection_path + '/' + filename        
        cmd = 'earthengine upload image --wait --asset_id={0} --nodata_value={1} {2}'\
            .format(filepath_ee, nodata_value, filepath_gs)        
        run(cmd)
        
        time_start = int(grids[0]['times'][0].timestamp() * 1000)
        cmd = 'earthengine asset set --time_start {0} {1}'\
            .format(time_start, filepath_ee)
        run(cmd)

        cmd = 'earthengine acl set public {0}'\
            .format(filepath_ee)
        run(cmd)
